In [21]:
import pandas as pd
from climada.engine import Impact
import numpy as np


In [2]:
emdat = pd.read_csv('/Users/szelie/OneDrive - ETH Zurich/20211208_Multi-Impact_ZelieCarmen/emdat_public_2022_03_03_query_uid-GxMNDW.csv')


In [3]:
emdat

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1980-9114-CIV,1980,9114,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.412386,NaN,NaN,NaN,NaN
1,1980-0096-DZA,1980,96,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,5200000.0,17098297.0,30.412386,NaN,NaN,NaN,NaN
2,1980-9114-GHA,1980,9114,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.412386,NaN,NaN,NaN,NaN
3,1980-9114-BEN,1980,9114,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,651.0,2141.0,30.412386,NaN,NaN,NaN,NaN
4,1980-9114-GMB,1980,9114,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.412386,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,2022-0061-REU,2022,61,NaN,Natural,Meteorological,Storm,Tropical cyclone,NaN,Cyclone 'Batsirai',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13916,2022-0077-ZAF,2022,77,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13917,2022-0108-ZAF,2022,108,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2708,NaN,Gauteng (Adm1).
13918,2022-0052-ZWE,2022,52,NaN,Natural,Meteorological,Storm,Tropical cyclone,NaN,Tropical storm 'Ana',...,NaN,NaN,NaN,NaN,NaN,NaN,1,3437;3438;3440;3441;69550,NaN,"Manicaland, Mashonaland Central, Mashonaland E..."


In [4]:
floods_region = emdat[(~emdat["Total Damages, Adjusted ('000 US$)"].isnull())&(~emdat["Geo Locations"].isnull())][emdat['Disaster Type']=='Flood']

/var/folders/f5/lms0b90j1kx7kk4v552qkjqw0000gp/T/ipykernel_24766/2503685359.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  floods_region = emdat[(~emdat["Total Damages, Adjusted ('000 US$)"].isnull())&(~emdat["Geo Locations"].isnull())][emdat['Disaster Type']=='Flood']


In [5]:
emdat = emdat[['Year', 'Disaster Type', 'ISO',"Total Damages, Adjusted ('000 US$)"]]


In [6]:
emdat = emdat.dropna()

In [7]:
emdat = emdat[emdat['Disaster Type'].isin(['Flood', 'Wildfire', 'Storm'])]

In [8]:
emdat = emdat.groupby(['ISO','Disaster Type','Year']).sum().reset_index()

In [9]:
emdat[emdat['ISO']=='TWN']

,ISO,Disaster Type,Year,"Total Damages, Adjusted ('000 US$)"
1853,TWN,Flood,1997,98417.0
1854,TWN,Flood,1998,31584.0
1855,TWN,Flood,2005,86026.0
1856,TWN,Flood,2006,156096.0
1857,TWN,Flood,2012,19946.0
1858,TWN,Flood,2017,31063.0
1859,TWN,Storm,1986,1198999.0
1860,TWN,Storm,1992,67590.0
1861,TWN,Storm,1994,835441.0
1862,TWN,Storm,1996,2245843.0


In [ ]:
emdat[emdat['ISO']=='TWN']

In [ ]:
impacts_emdat = {}
for country in emdat['ISO']:
    impacts_emdat[country]={}
    for hazard in ['Flood', 'Storm', 'Wildfire']:
        impact = Impact()
        impact.at_event = np.array(emdat[(emdat['ISO']==country)&(emdat['Disaster Type']==hazard)]["Total Damages, Adjusted ('000 US$)"])
        #impact.date = impacts_yearsets['TWN']['RF'][0].date[0:len(impact.at_event)]
        impact.frequency = np.ones(len(impact.at_event))/len(impact.at_event)
        impact.aai_agg = impact.aai_agg_from_at_event(impact.at_event,impact.frequency)
        impacts_emdat[country][hazard] = impact